In [6]:
import yaml
import json
import vocab_build_tools as tools

stream = open('./config.yaml', 'r')

documents = yaml.load_all(stream, Loader=yaml.FullLoader)

for doc in documents:
    merged_df = tools.create_df(doc['termLists'])
    config = doc

examples = merged_df[~merged_df['examples'].isna()]

classes = ['TaxonConcept', 'TaxonConceptMappings', 'TaxonName', 'NomenclaturalType']

header_object = open('static/examples-header.md', 'rt', encoding='utf-8')
header = header_object.read()
header_object.close()

with open('../docs/examples/index.md', 'w') as f:
    
    f.write(header)
    
    for cl in classes:
        f.write("\n## " + cl + "\n\n")
        
        for index, row in examples[examples['organizedInClass'] == 'http://rs.tdwg.org/tcs/terms/' + cl].iterrows():
            alias = [n['vann_preferredNamespacePrefix'] for n in config['termLists'] if n['vann_preferredNamespaceUri'] == row['namespace']][0]
            
            if type(row['examples']) == list:
                f.write("\n### " + alias + ":" + row['localName'] + "\n\n")
                for ex in row['examples']:
                    f.write(f"- [{ex}]({ex}.md)\n")
                    # text = tools.add_example(ex)
                    # f.write(text)


In [9]:
import yaml
import json
import vocab_build_tools as tools

stream = open('./config.yaml', 'r')

documents = yaml.load_all(stream, Loader=yaml.FullLoader)

for doc in documents:
    merged_df = tools.create_df(doc['termLists'])
    config = doc

examples = merged_df[~merged_df['examples'].isna()]

classes = ['TaxonConcept', 'TaxonConceptMappings', 'TaxonName', 'NomenclaturalType']

for cl in classes:
    for index, row in examples[examples['organizedInClass'] == 'http://rs.tdwg.org/tcs/terms/' + cl].iterrows():
        alias = [n['vann_preferredNamespacePrefix'] for n in config['termLists'] if n['vann_preferredNamespaceUri'] == row['namespace']][0]
        
        if type(row['examples']) == list:
            for index, ex in enumerate(row['examples']):
                with open('../docs/examples/' + ex + '.md', 'w') as f:
                    heading = ex.replace('-', ' ')
                    f.write('# ' + heading + '\n')
                    f.write('\n\n')
                    f.write('**Term:** ')
                    f.write('[' + alias + ':' + row['localName'] + '](/terms/#' + alias + '_' + row['localName'].lower() + ')')
                    f.write('\n\n')

                    if (len(row['examples']) > 1):
                        exs = []
                        for i, exx in enumerate(row['examples']):
                            if i == index:
                                exs.append(exx)
                            else:
                                exs.append(f'[{exx}](./{exx}.html)')
                        f.write(' | '.join(exs))

                    text = tools.add_example(ex)
                    f.write(text)


In [ ]:
import yaml
import vocab_build_tools as tools

stream = open('./config.yaml', 'r')

documents = yaml.load_all(stream, Loader=yaml.FullLoader)

for doc in documents:
    merged_df = tools.create_df(doc['termLists'])
    config = doc

examples = merged_df[~merged_df['examples'].isna()]

namespaces = []
aliases = []
for row in examples['examples'].to_list():
    if type(row) == list:
        for ex in row:
            path = '../examples/' + ex + '.ttl'
            with open(path, 'r') as f:
                for line in f:
                    if line[:7] == '@prefix':
                        ns = line[8:-2]
                        alias, uri = ns.split(': ', 1)
                        if alias not in aliases:
                            aliases.append(alias)
                            namespaces.append({
                                'alias': alias,
                                'uri': uri.strip(' <>')
                            })
                            
table = '| alias | uri |\n'
table += '|-|-|\n'
for ns in namespaces:
    table += '| ' + ns['alias'] + ' | ' + ns['uri'] + ' |\n'
    
print(table)